In [ ]:
#### TensorFlow

In [ ]:
from __future__ import absolute_import, division, print_function

!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import tensorflow_datasets as tfds

In [ ]:
# install some libraries if not installed already
!pip install --upgrade scikit-image 
!pip install imgaug
!pip install pandas==0.23.4
!pip install pydicom
!pip install seaborn==0.9.0

In [ ]:
# import some libraries we will use
import os, sys, glob, csv, random
import pydicom

import matplotlib.pyplot as plt
import numpy as np

from skimage import io

import seaborn as sns
import pandas as pd

from PIL import Image

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
# MNIST handwritten digits

In [ ]:
# get the dataset if you don't have it already or on Colab - 1
!git clone https://github.com/myleott/mnist_png.git

In [ ]:
# get the dataset if you don't have it already or on Colab - 2
os.chdir('mnist_png')

In [ ]:
# get the dataset if you don't have it already or on Colab - 3
!tar -xzf mnist_png.tar.gz
!ls

In [ ]:
mnist_base_dir_tr = 'mnist_png' # or point to appropriate directory if you're on Biowulf and already have the data

In [ ]:
# load all images in the training set
mnist_digits = dict.fromkeys(range(10))
for num in range(10):
    mnist_digits[num] = glob.glob(os.path.join(mnist_base_dir_tr, 'training', str(num), '*.png'))

In [ ]:
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=1)
  image = tf.image.resize(image, [28, 28])

  return image

In [ ]:
def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  return preprocess_image(image)

In [ ]:
all_image_paths = []
all_image_labels = []
for num in range(10):
  all_image_paths += mnist_digits[num]
  all_image_labels += [num]*len(mnist_digits[num])

In [ ]:
ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))

# The tuples are unpacked into the positional arguments of the mapped function 
def load_and_preprocess_from_path_label(path, label):
  return load_and_preprocess_image(path), label

image_label_ds = ds.map(load_and_preprocess_from_path_label)
image_label_ds

In [ ]:
BATCH_SIZE = 64

ds = image_label_ds.apply(
  tf.data.experimental.shuffle_and_repeat(buffer_size=len(all_image_paths)))
ds = ds.batch(BATCH_SIZE)
ds = ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
model = models.Sequential()
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])

In [ ]:
model.fit(ds, epochs=10, steps_per_epoch=3)

In [ ]:
#### PyTorch

In [ ]:
# install PyTorch if not installed already
!pip install torch torchvision

In [ ]:
# import PyTorch and related libraries
import torch 
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms, datasets
from torch.utils.data.dataset import Dataset
import torch.nn.parallel

In [ ]:
data_transform = transforms.Compose([
        transforms.Resize(28),
        transforms.ToTensor(),
    ])

In [ ]:
class MNIST_Dataset(Dataset):
    '''RSNA Pneumonia Dataset'''
    def __init__(self, image_paths, labels, transform=None):
        
        self.X = np.asarray(image_paths)
        self.Y = np.asarray(labels)
        
        self.transform = transform;
        
    def __len__(self):
        return len(self.X);
    
    def __getitem__(self, idx):
        item = Image.open(self.X[idx])
        label = self.Y[idx];
        
        if self.transform:
            item = self.transform(item);
        
        return (item, label)

In [ ]:
# train configs
num_epochs = 10
batch_size = 64
learning_rate = 0.001
num_workers = 4
num_gpus = 1
device = torch.device('cuda')

In [ ]:
train_dataset = MNIST_Dataset(all_image_paths, all_image_labels, data_transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=num_workers)

In [ ]:
# network with a fully connected layer
class FC(nn.Module):
    def __init__(self):
        super(FC, self).__init__()
        self.fc = nn.Linear(28*28, 10)
        
    def forward(self, x):
        out = x
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
# define network and optimizer
fc = FC().to(device);

criterion = nn.CrossEntropyLoss();
optimizer = torch.optim.Adam(fc.parameters(), lr=learning_rate);

In [ ]:
# train
losses = [];
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.float()).to(device)
        labels = Variable(labels).to(device)

        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = fc(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item());
        
        if (i+1) % 10 == 0:
            print ('Epoch : %d/%d, Iter : %d/%d,  Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.item()))

In [ ]:
# report accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in train_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = fc(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy: %d %%' % (
    100 * correct / total))

In [ ]:
# Exercise 1: Try to improve the performance of the networks.
# What are the first obvious things to do, and what can you do more?

In [ ]:
# Exercise 2: Try to do the same for the chest x-ray dataset with three classes